In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key=' your api key', 
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-51404?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer I, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer I, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does 

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [11]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer I, ITC',
 'experience': '1+ years of professional experience as a Full Stack Software Engineer or in a similar role',
 'skills': ['Front-End Skills in React or similar technologies',
  'Server-side programming languages such as Node.js, Python or similar',
  'NoSQL databases',
  'API Development: designing and consuming RESTful APIs and web services',
  'Cloud Technologies: familiarity with cloud platforms preferably AWS',
  'Version Control: proficient with Git, GitHub, or similar version control tools',
  'Testing Frameworks: experience with unit testing, integration testing, and test-driven development (TDD)'],
 'description': 'Design & Develop: Build, test, and maintain scalable web applications and services. Collaborate: Work closely with product managers, designers, and other engineers to define requirements, technical specifications, and deliver quality software.'}

In [12]:
type(json_res)

dict

In [13]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [14]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [19]:
links = collection.query(query_texts=['Full Stack '], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [21]:
job = json_res
job['skills']

['Front-End Skills in React or similar technologies',
 'Server-side programming languages such as Node.js, Python or similar',
 'NoSQL databases',
 'API Development: designing and consuming RESTful APIs and web services',
 'Cloud Technologies: familiarity with cloud platforms preferably AWS',
 'Version Control: proficient with Git, GitHub, or similar version control tools',
 'Testing Frameworks: experience with unit testing, integration testing, and test-driven development (TDD)']

In [22]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
         You are shreya, a final year student pursuing a bachelor's degree in Computer Science. 
    Write a cold email regarding the job mentioned above.
    Highlight your interest in applying your AI and full-stack development skills to this role. 
    Mention your relevant skills and enthusiasm for the position.
    Also, include the most relevant links from the following to showcase your projects or experiences: {link_list}
    Keep the tone professional and concise.  
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Services for Your Business Needs

Dear Hiring Manager,

I came across the job description for a Software Engineer I, ITC, and I believe AtliQ, an AI & Software Consulting company, can fulfill your requirements. With over 1+ years of experience in delivering tailored software solutions, we can empower your enterprise with scalable, efficient, and cost-effective solutions.

Our team of expert software engineers possesses the necessary skills to design, develop, and maintain scalable web applications and services. We have hands-on experience with Front-End Skills in React, server-side programming languages such as Node.js and Python, NoSQL databases, API Development, Cloud Technologies (preferably AWS), Version Control (Git, GitHub), and Testing Frameworks (unit testing, integration testing, and TDD).

At AtliQ, we have a proven track record of collaborating with clients to define requirements, technical specifications, and deliver quality software. Ou